#### Algorithm.
Detecting infinite loops and pairing up elements in a list to maximize the occurences of infinite loops. Was much easier than initially thought. Even the naive approach worked! 



In [1]:
""" Challenge 4.2. Loop maximization problem """

import numpy as np, itertools as it

def thumb_wrestling(pair, progress = False):
    if progress:    print("Start: banana pair = ", pair)
    
    ## If the sum of bananas is odd, the participants will 
    ## never have equal number of bananas... 
    if sum(pair) % 2:
        if progress:    print("Seems like this game will go on forever!")
        return True
    
    ## If the sum of bananas is even, the game can stop at one point...
    rounds, GCD = 0, [np.gcd(*pair)]
    while pair[0] != pair[1]:
        rounds += 1;    bet, win = min(pair), int(pair[0] < pair[1])
        pair = (pair[0] - (-1)**win * bet, pair[1] + (-1)**win * bet)
        GCD.append(np.gcd(*pair))
        if progress:    print("Round %d: banana pair = %s, \tGCD = %d" % (rounds, str(pair), GCD[-1]))
        
        ## For a looping game, the GCD of the two numbers will become fixed after 1/2 rounds- 
        ## while for a terminating game, it will be doubled in each round...
        if sum(GCD[-1] == np.array(GCD)) > 3:
            if progress:    print("Seems like this game will go on forever!")
            return True
    if progress:    print("Game over! Total rounds played =", rounds)
    return False

def solution(banana_list):
    guards = len(banana_list)

    ## Count loops for each guard & store loop partners...
    loops, partners = [0 for _ in range(guards)], [[ ] for _ in range(guards)]
    for i, j in it.combinations(range(guards), 2):
        if thumb_wrestling((banana_list[i], banana_list[j]), progress = False):
            loops[i] += 1;    partners[i].append(j)
            loops[j] += 1;    partners[j].append(i)
    sorted_guards = sorted(range(guards), key = lambda gg: loops[gg])    # Sort by loop counts
    
    ## Get best pairings & count occupied guards...
    ##  Start with the guard with lowest loop count, pair him/her with the partner 
    ##  with highest loop count from available partners (not in another game already)
    pairings, occupied = [ ], [ ]
    for guard in sorted_guards:
        if len(occupied) == guards:    break    
        if guard not in occupied:
            available = [pp for pp in partners[guard] if pp not in occupied]
            if len(available):
                if len(available) > 1:
                    available = sorted(available, key = lambda pp: -loops[pp])
                pairings.append((guard, available[0]))
                occupied += list(pairings[-1])
    left = guards - len(occupied)
    return left


In [2]:
## Examples...
%time print("Number of guards left to watch the prisoners = ", solution(banana_list = [1, 1]))
%time print("Number of guards left to watch the prisoners = ", solution(banana_list = [1, 7, 3, 21, 13, 19]))
%time print("Number of guards left to watch the prisoners = ", solution(banana_list = [1, 7, 5, 3, 19, 13, 53, 61]))
%time print("Number of guards left to watch the prisoners = ", solution(banana_list = [1, 3, 3, 1, 5, 1]))


Number of guards left to watch the prisoners =  2
Wall time: 0 ns
Number of guards left to watch the prisoners =  0
Wall time: 998 µs
Number of guards left to watch the prisoners =  0
Wall time: 1.99 ms
Number of guards left to watch the prisoners =  4
Wall time: 0 ns
